In [2]:
import pandas as pd
import numpy as np

ny_path = "/home/fuest/EnData/data/pecanstreet/15minute_data_newyork.csv"
austin_path = "/home/fuest/EnData/data/pecanstreet/15minute_data_austin.csv"
cali_path = "/home/fuest/EnData/data/pecanstreet/15minute_data_california.csv"
metadata_path = "/home/fuest/EnData/data/pecanstreet/metadata.csv"

ny_data = pd.read_csv(ny_path)
austin_data = pd.read_csv(austin_path)
cali_data = pd.read_csv(cali_path)
metadata = pd.read_csv(metadata_path)

ny_user_ids = ny_data.dataid.unique().tolist()
austin_user_ids = austin_data.dataid.unique().tolist()
cali_user_ids = cali_data.dataid.unique().tolist()

In [3]:
all_user_ids = cali_user_ids + austin_user_ids + ny_user_ids

In [4]:
metadata = metadata.loc[metadata["dataid"].isin(all_user_ids)].copy()
metadata["car1"].value_counts()

yes    15
Name: car1, dtype: int64

In [ ]:

USER_IDs = [27,  142,  203,  387,  558,  661,  914,  950, 1222, 1240, 1417,
       1450, 1524, 1642, 1731, 2096, 2318, 2335, 2358, 2361, 2606, 2818,
       3000, 3039, 3456, 3488, 3517, 3538, 3687, 3700, 3864, 3938, 3996,
       4031, 4283, 4373, 4495, 4550, 4767, 4934, 5058, 5587, 5679, 5746,
       5938, 5982, 5997, 6139, 6377, 6547, 7062, 7114, 7536, 7719, 7800,
       7901, 7951, 8061, 8156, 8342, 8386, 8565, 8574, 8733, 9019, 9053,
       9160, 9213, 9278, 9612, 9775, 9836, 9922]


In [1]:
from datasets.pecanstreet import PecanStreetDataset

data = PecanStreetDataset(normalize=True, include_generation=False)
user_data = data.create_user_dataset(661)

KeyError: 'dataid'